In [ ]:
import urllib.request
import pandas as pd
import json
import numpy as np
import networkx as nx
import time
import pickle
import matplotlib.pyplot as plt
import urllib.parse


In [ ]:
GN = nx.read_gml("/content/drive/My Drive/Colab Notebooks/SG/final project/gamesNetConnectedCommontagsProportional.gml")

In [ ]:
print(len(GN))

3780


In [ ]:
#https://store.steampowered.com/appreviews/440?json=1&cursor=AoIIPwFEZ37A2asC&language=english&num_per_page=100
baseurl = "https://store.steampowered.com/appreviews/"
format ="json=1"
language = "language=english"
num_per_page = "num_per_page=100"

In [ ]:
PAY ATTENTION!!!
dict_comments = {}
#with open ('/content/drive/My Drive/Colab Notebooks/SG/final project/comm_dict_300.p', 'rb') as fp:
#    dict_comments = pickle.load(fp)

In [ ]:
done = list(dict_comments.keys())
to_be_do = [node for node in GN.nodes() if node not in done]

In [ ]:
print(done[0:10])
print(len(done))
print(to_be_do[0:10])
print(len(to_be_do))

['730', '570', '578080', '945360', '271590', '1172470', '252950', '440', '359550', '346110']
3780
[]
0


In [ ]:
#CAMBIA GN.nodes() CON to_be_do
i=0
for node in to_be_do:
  print(i)
  i+=1
  cursor = "cursor=*"
  appid = node
  req = "{}{}?{}&{}&{}&{}".format(baseurl, appid, format, language, num_per_page, cursor)
  steamresponse = urllib.request.urlopen(req)
  steamdata = steamresponse.read()
  steamtext = steamdata.decode('utf-8')
  content = json.loads(steamtext)
  if content["success"]==False:
    continue
  else:
    dict_comments[appid] = []
    retrieved_comments = content["query_summary"]["num_reviews"]
    total_reviews = content["query_summary"]["total_reviews"]
    for rev in content["reviews"]:
      dict_comments[appid].append( { "weight" : rev["weighted_vote_score"], "comment" : rev["review"] } )

  while retrieved_comments < total_reviews and retrieved_comments < 300 and cursor!=content["cursor"]:
    cursor=content["cursor"]
    req = "{}{}?{}&{}&{}&{}".format(baseurl, appid, format, language, num_per_page, "cursor="+urllib.parse.quote(cursor, safe=''))
    steamresponse = urllib.request.urlopen(req)
    steamdata = steamresponse.read()
    steamtext = steamdata.decode('utf-8')
    content = json.loads(steamtext)
    if content["success"]==False:
      break
    else:
      retrieved_comments += content["query_summary"]["num_reviews"]
      for rev in content["reviews"]:
        dict_comments[appid].append( { "weight" : rev["weighted_vote_score"], "comment" : rev["review"] } )


In [ ]:

print(len(dict_comments))

3780


In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/SG/final project/comm_dict_300.p', 'wb') as fp:
    pickle.dump(dict_comments, fp)

#with open ('/content/drive/My Drive/Colab Notebooks/SG/final project/responses.p', 'rb') as fp:
#    responses = pickle.load(fp)

In [ ]:
cursor = "cursor=*"
appid = "730"
req = "{}{}?{}&{}&{}&{}".format(baseurl, appid, format, language, num_per_page, cursor)
steamresponse = urllib.request.urlopen(req)
steamdata = steamresponse.read()
steamtext = steamdata.decode('utf-8')
content = json.loads(steamtext)

In [ ]:
print(content["reviews"][0])

{'recommendationid': '78046053', 'author': {'steamid': '76561199099667228', 'num_games_owned': 99, 'num_reviews': 2, 'playtime_forever': 4812, 'playtime_last_two_weeks': 4530, 'playtime_at_review': 677, 'last_played': 1604660640}, 'language': 'english', 'review': 'great game, lots of cheaters.', 'timestamp_created': 1603501422, 'timestamp_updated': 1603501422, 'voted_up': True, 'votes_up': 178, 'votes_funny': 28, 'weighted_vote_score': '0.880124032497406006', 'comment_count': 1, 'steam_purchase': True, 'received_for_free': False, 'written_during_early_access': False}


In [ ]:
#success, query_summary->(num_reviews, total_reviews), reviews[i]->(review, weighted_vote_score), cursor

In [ ]:
print(type(steamtext))

<class 'str'>
